# IMPORTS

In [ ]:
import numpy as np
import seaborn as sns
import copy
import scipy.optimize

import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

# FRAMEWORK

In [ ]:
class Coordinate():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f'({self.x}, {self.y})'
        
        
    def move(self, direction, probs):
        p = np.random.uniform()
        
        if direction == 3 and self.y < NROWS-1 and p > probs[3]:
            self.y += 1
        elif direction == 2 and self.y > 0 and p > probs[2]:
            self.y -= 1
        elif direction == 1 and self.x < NCOLUMNS-1 and p > probs[1]:
            self.x += 1
        elif direction == 0 and self.x > 0 and p > probs[0]:
            self.x -= 1
            
    def equals(self, coordinate):
        return (self.x == coordinate.x and self.y == coordinate.y)
    
    def copy(self):
        return Coordinate(self.x, self.y)
    
    def get_distance(self, coordinate):
        return np.sqrt((self.x - coordinate.x)**2 + (self.y - coordinate.y)**2)
        
    
class Gridworld():
    def __init__(self, goal=Coordinate(0, 9)):
        self.rows = NROWS
        self.columns = NCOLUMNS
        self.goal = goal
        self.transitions = np.around(np.random.uniform(size=(NROWS, NCOLUMNS, 4)), 1)
        self.qtable = np.zeros((NROWS, NCOLUMNS, 4))
        
        
    def reset_qtable(self):
        self.qtable = np.zeros((NROWS, NCOLUMNS, 4))

            
    def use_heuristic(self, start):
        current_state = start.copy()
        steps = 0
        
        while (not current_state.equals(self.goal)) and (steps <= 500):
            move = self.get_heuristic_move(current_state)
            current_state.move(move, self.transitions[current_state.y, current_state.x, :])
            steps += 1
        
        return steps
    
    
    def get_heuristic_move(self, current_state):
        if current_state.x > self.goal.x:  # West
            return 0
        elif current_state.x < self.goal.x:  # East
            return 1
        
        if current_state.y > self.goal.y:  # North
            return 2
        elif current_state.y < self.goal.y:  # South
            return 3
        
        return np.random.choice(options)
        
        
    def q_learning(self, start, gamma=0.99, epsilon=0.05, alpha=0.1, max_steps=100, method='regular'):
        current_state = start.copy()
        steps = 0
        while (not current_state.equals(self.goal)) and (steps < max_steps):

            move = np.random.choice(np.argwhere(self.qtable[current_state.y, current_state.x, :] == 
                                                self.qtable[current_state.y, current_state.x, :].max()).flatten())

            if np.random.uniform() < epsilon:
                move = np.random.choice([0, 1, 2, 3])
                
            next_state = current_state.copy()
            next_state.move(move, self.transitions[next_state.y, next_state.x, :])

            reward = self.get_reward(next_state, current_state, method)

            self.qtable[current_state.y, current_state.x, move] = round(self.qtable[current_state.y, current_state.x, move] + \
                                               alpha*(reward + gamma*self.qtable[next_state.y, next_state.x, :].max() \
                                               - self.qtable[current_state.y, current_state.x, move]), 4)
                
            steps += 1
            current_state = next_state.copy()
        
        return self.qtable        
        
    
    def get_reward(self, current_state, previous_state, method):
        if method == 'regular':
            return 1 if current_state.equals(self.goal) else -1
        elif method == 'modified':
            if current_state.equals(self.goal):
                return 1
            else:
                return -current_state.get_distance(self.goal)/max_d
    
    
    def value_iteration(self):
        values = np.zeros((NROWS, NCOLUMNS))
        rewards = np.ones((NROWS, NCOLUMNS))*-1
        rewards[self.goal.y, self.goal.x] = 1
        steps = 0
        
        while True:
            new_values = np.zeros((NROWS, NCOLUMNS, 4))
        
            new_values[:, :, 0] = (((1-self.transitions[:, :, 0]) * (np.c_[np.ones(NROWS)*-1, rewards][:, :-1] + 
                                                    np.c_[np.ones(NROWS)*-1, values][:, :-1])) +
                                  (self.transitions[:, :, 0] * (rewards + values)))

            new_values[:, :, 1] = (((1-self.transitions[:, :, 1]) * (np.c_[rewards, np.ones(NROWS)*-1][:, 1:] + 
                                                    np.c_[values, np.ones(NROWS)*-1][:, 1:])) +
                                  (self.transitions[:, :, 1] * (rewards + values)))

            new_values[:, :, 2] = (((1-self.transitions[:, :, 2]) * (np.r_['0,2', np.ones(NCOLUMNS)*-1, rewards][:-1, :] + 
                                                    np.r_['0,2', np.ones(NCOLUMNS)*-1, values][:-1, :])) +
                                  (self.transitions[:, :, 2] * (rewards + values)))

            new_values[:, :, 3] = (((1-self.transitions[:, :, 3]) * (np.r_['0,2', rewards, np.ones(NCOLUMNS)*-1][1:, :] + 
                                                    np.r_['0,2', values, np.ones(NCOLUMNS)*-1][1:, :])) +
                                  (self.transitions[:, :, 3] * (rewards + values)))
        
            
            max_new_values = new_values.max(axis=2)
            max_new_values -= max_new_values.min()
            
            if np.abs(values - max_new_values).max() < 1e-8:
                break
            else:
                values = copy.deepcopy(max_new_values)
                steps += 1
        
        print(f'Value iteration converged after {steps} steps')        
        return values - values.max() + 1, new_values.argmax(axis=2)
    
    
    def eval_policy(self, policy, start):
        steps = 0
        current_state = start.copy()
        while (not current_state.equals(self.goal)) and (steps <= 500):
            move = policy[current_state.y, current_state.x]            
            current_state.move(move, self.transitions[current_state.y, current_state.x, :])
            steps += 1
        return steps
        
    
    def plot_policy(self, values, policy, name=None, complete=True):
        values = (values - values.min()) / (values.max() - values.min())
        
        probs = np.zeros((policy.shape[0], policy.shape[1]))
        for i in range(policy.shape[0]):
            for j in range(policy.shape[1]):
                probs[i, j] = 1 - self.transitions[i, j, policy[i, j]]
                
        X, Y = np.meshgrid(np.arange(NCOLUMNS)+0.5, np.arange(NROWS)+0.5)
        U = np.where(policy == 1, 1, np.where(policy == 0, -1, 0))
        V = np.where(policy == 2, -1, np.where(policy == 3, 1, 0))

        fig, ax = plt.subplots(figsize=(10,10))        
        sns.heatmap(values, ax=ax, cmap='gray')
        ax.quiver(X, Y, U, V, probs, edgecolors='w', cmap='RdYlGn')
        
        ax.invert_yaxis()
        ax.set_xlabel('X-Coordinate', fontsize=15)
        ax.set_ylabel('Y-Coordinate', fontsize=15)
        fig.show()
        
        if name is not None:
            fig.savefig(name)
            fig, ax = plt.subplots(figsize=(10,10))       
            
            halfrows = int(NROWS/2)
            halfcolumns = int(NCOLUMNS/2)
            policy_part = policy[:halfrows, :halfcolumns]
            value_part = values[:halfrows, :halfcolumns]

            probs = np.zeros((policy_part.shape[0], policy_part.shape[1]))
            for i in range(policy_part.shape[0]):
                for j in range(policy_part.shape[1]):
                    probs[i, j] = 1 - self.transitions[i, j, policy_part[i, j]]
                    
            X, Y = np.meshgrid(np.arange(halfcolumns)+0.5, np.arange(halfrows)+0.5)
            U = np.where(policy_part == 1, 1, np.where(policy_part == 0, -1, 0))
            V = np.where(policy_part == 2, -1, np.where(policy_part == 3, 1, 0))


            sns.heatmap(value_part, ax=ax, cmap='gray')
            ax.quiver(X, Y, U, V, probs, edgecolors='w', cmap='RdYlGn')
            
            ax.invert_yaxis()
            ax.set_xlabel('X-Coordinate', fontsize=15)
            ax.set_ylabel('Y-Coordinate', fontsize=15)
            
            fig.savefig(name+'_part')
                

# ENVIRONMENT

In [ ]:
np.random.seed(26)

NROWS = 50
NCOLUMNS = 50
max_d = np.sqrt(NROWS**2 + NCOLUMNS**2)
its = 10

env = Gridworld(Coordinate(0, 9))

In [ ]:
trainset = np.array([
    Coordinate(5, 10),
    Coordinate(10, 15),
    Coordinate(10, 5),
    Coordinate(25, 10),
    Coordinate(15, 20),
    Coordinate(15, 0),
    Coordinate(15, 10),
    Coordinate(0, 20),
    Coordinate(20, 25),
    Coordinate(5, 30),
    Coordinate(35, 5),
    Coordinate(35, 25),
    Coordinate(10, 40),
    Coordinate(25, 35),
    Coordinate(30, 40),
    Coordinate(30, 15),
    Coordinate(15, 35),
    Coordinate(45, 15),
    Coordinate(20, 45),
    Coordinate(40, 30),
    Coordinate(40, 40),
    Coordinate(45, 5),
    Coordinate(5, 45),
    Coordinate(35, 45),
    Coordinate(45, 45)
], dtype=Coordinate)


distances = np.zeros(len(trainset))

for i, start in enumerate(trainset):  
    distance = start.get_distance(env.goal)
    distances[i] = distance

trainset = trainset[np.argsort(distances)]
distances = np.sort(distances)

mean_steps_taken = np.zeros((4, len(trainset)))
std_steps_taken = np.zeros((4, len(trainset)))

In [ ]:
start_grid = np.zeros((NROWS, NCOLUMNS))

for c in trainset:
    start_grid[c.y, c.x] = 1

start_grid[env.goal.y, env.goal.x] = -1

fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(start_grid, ax=ax, cmap='gray')
ax.invert_yaxis()

# HEURISTIC

In [ ]:
steps = np.zeros((its, len(trainset)))
for i, start in enumerate(trainset):
    for j in range(its):
        step = env.use_heuristic(start)
        steps[j, i] = step
        print ("\r [{}/{}][{}/{}]".format(i+1, len(trainset), j+1, its), end="")

mean_steps_taken[0, :] = steps.mean(axis=0)
std_steps_taken[0, :] = steps.std(axis=0)

# VALUE ITERATION (DYNAMIC PROGRAMMING)

In [ ]:
values_DP, policy_DP = env.value_iteration()

In [ ]:
steps = np.zeros((its, len(trainset)))
for i, start in enumerate(trainset):
    for j in range(its):
        step = env.eval_policy(policy_DP, start)
        steps[j, i] = step
        print ("\r [{}/{}][{}/{}]".format(i+1, len(trainset), j+1, its), end="")

mean_steps_taken[1, :] = steps.mean(axis=0)
std_steps_taken[1, :] = steps.std(axis=0)

In [ ]:
env.plot_policy(values_DP, policy_DP, 'Value Iteration')

# SYSTEM OF EQUATIONS

In [ ]:
rewards = np.ones((NROWS, NCOLUMNS))*-1
rewards[env.goal.y, env.goal.x] = 1


def f(values, policy = False):
    values = values.reshape(NROWS, NCOLUMNS)
    
    new_values = np.zeros((NROWS, NCOLUMNS, 4))
    new_values[:, :, 0] = (((1-env.transitions[:, :, 0]) * (np.c_[np.ones(NROWS)*-1, rewards][:, :-1] + 
                                                    np.c_[np.ones(NROWS)*-1, values][:, :-1])) +
                                  (env.transitions[:, :, 0] * (rewards + values)))

    new_values[:, :, 1] = (((1-env.transitions[:, :, 1]) * (np.c_[rewards, np.ones(NROWS)*-1][:, 1:] + 
                                                    np.c_[values, np.ones(NROWS)*-1][:, 1:])) +
                                  (env.transitions[:, :, 1] * (rewards + values)))

    new_values[:, :, 2] = (((1-env.transitions[:, :, 2]) * (np.r_['0,2', np.ones(NCOLUMNS)*-1, rewards][:-1, :] + 
                                                    np.r_['0,2', np.ones(NCOLUMNS)*-1, values][:-1, :])) +
                                  (env.transitions[:, :, 2] * (rewards + values)))

    new_values[:, :, 3] = (((1-env.transitions[:, :, 3]) * (np.r_['0,2', rewards, np.ones(NCOLUMNS)*-1][1:, :] + 
                                                    np.r_['0,2', values, np.ones(NCOLUMNS)*-1][1:, :])) +
                                  (env.transitions[:, :, 3] * (rewards + values)))


    max_new_values = new_values.max(axis=2)
    max_new_values -= max_new_values.min()
    
    if policy:
        return max_new_values, new_values.argmax(axis=2)
    return -(values - max_new_values).reshape(-1)

In [ ]:
x = scipy.optimize.excitingmixing(f, np.ones((NROWS, NCOLUMNS)), iter=1500)
values_EQ, policy_EQ = f(x, policy=True)

In [ ]:
env.plot_policy(values_EQ, policy_EQ, 'System of Equations')

In [ ]:
(policy_DP == policy_EQ).sum() /(50*50)

# Q-LEARNING SET-UP

In [ ]:
n_its = 1500
n_steps = 250

# Q-LEARNING (REGULAR)

In [ ]:
env.reset_qtable()
for i, start in enumerate(trainset):
    for j in range(n_its):
        env.q_learning(start, max_steps=n_steps)
        print ("\r [{}/{}][{}/{}]".format(i+1, len(trainset), j+1, n_its), end="")

values_Qreg, policy_Qreg = env.qtable.max(axis=2), env.qtable.argmax(axis=2)

In [ ]:
steps = np.zeros((its, len(trainset)))
for i, start in enumerate(trainset):
    for j in range(its):
        step = env.eval_policy(policy_Qreg, start)
        steps[j, i] = step
    print ("\r Iteration: {}".format(i+1), end="")

mean_steps_taken[2, :] = steps.mean(axis=0)
std_steps_taken[2, :] = steps.std(axis=0)

In [ ]:
env.plot_policy(values_Qreg, policy_Qreg, 'Q-learning (reg)')

# Q-LEARNING (MODIFIED)

In [ ]:
env.reset_qtable()
for i, start in enumerate(trainset):
    for j in range(n_its):
        env.q_learning(start, max_steps=n_steps, method='modified')
        print ("\r [{}/{}][{}/{}]".format(i+1, len(trainset), j+1, n_its), end="")

values_Qmod, policy_Qmod = env.qtable.max(axis=2), env.qtable.argmax(axis=2)

In [ ]:
steps = np.zeros((its, len(trainset)))
for i, start in enumerate(trainset):
    for j in range(its):
        step = env.eval_policy(policy_Qmod, start)
        steps[j, i] = step
    print ("\r Iteration: {}".format(i+1), end="")

mean_steps_taken[3, :] = steps.mean(axis=0)
std_steps_taken[3, :] = steps.std(axis=0)

In [ ]:
env.plot_policy(values_Qmod, policy_Qmod, 'Q-learning (mod)')

# COMPARISON

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 

ax.plot(distances, mean_steps_taken[0], 'o', c='red', label='Heuristic', ms=10)
ax.plot(distances, mean_steps_taken[1], 'v', c='blue', label='Value Iteration', ms=10)
ax.plot(distances, mean_steps_taken[2], 's', c='green', label='Q-learning (reg)', ms=7)
ax.plot(distances, mean_steps_taken[3], 'p', c='magenta', label='Q-learning (mod)', ms=7)


ax.grid()
ax.legend(loc = 'center right', prop={'size': 12})
ax.set_xlabel('Distance from goal', fontsize=15)
ax.set_ylabel('Average steps needed', fontsize=15)
fig.savefig('comparison')
fig.show()

In [ ]:
(policy_DP == policy_Qmod).sum() / (NROWS*NCOLUMNS)

In [ ]:
(policy_DP == policy_Qreg).sum() / (NROWS*NCOLUMNS)

In [ ]:
(policy_Qmod == policy_Qreg).sum() / (NROWS*NCOLUMNS)